In [1]:
import os
import pickle

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Conv1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import set_random_seed
import pandas as pd
import numpy as np

In [2]:
# check if tensorflow sees GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# seed for replicibility of results
set_random_seed(823)

In [4]:
# load train and validation labels
df_train = pd.read_csv('train_labels.csv')
df_val = pd.read_csv('validation_labels.csv')

In [5]:
# append train features and target data to lists
train_frames = []
train_class = []
for file in os.listdir('train'):
    df_temp = pd.read_csv(f'train/{file}')
    df_temp = df_temp.iloc[1500:3001, 1:]
    df_temp = df_temp.to_numpy()
    train_frames.append(df_temp)
    train_class.append(df_train[df_train['filename']==file]['class'])

In [6]:
# append validation features and target data to lists
val_frames = []
val_class = []
for file in os.listdir('validation'):
    df_temp = pd.read_csv(f'validation/{file}')
    df_temp = df_temp.iloc[1500:3001, 1:]
    df_temp = df_temp.to_numpy()
    val_frames.append(df_temp)
    val_class.append(df_val[df_val['filename']==file]['class'])

In [7]:
# init OneHotEncoder
ohe = OneHotEncoder(sparse_output=False).fit(train_class)

In [8]:
# save OneHotEncoder
with open('ohe.save', 'wb') as fp:
    pickle.dump(ohe, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# transform targets
train_class = ohe.transform(train_class)
val_class = ohe.transform(val_class)

In [10]:
# change lists to numpy arrays
train_frames = np.array(train_frames)
val_frames = np.array(val_frames)

In [11]:
# init MinMaxScaler
mms = MinMaxScaler().fit(train_frames[0])

In [12]:
# save MinMaxScaler
with open('mms.save', 'wb') as fp:
    pickle.dump(mms, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# transform data for every subject
for x in range(train_frames.shape[0]):
    train_frames[x] = mms.transform(train_frames[x])

for x in range(val_frames.shape[0]):
     val_frames[x] = mms.transform(val_frames[x])

In [14]:
# make model
model = Sequential([
    Conv1D(10, 3),
    BatchNormalization(),
    Conv1D(20, 3),
    BatchNormalization(),
    Conv1D(30, 3),
    BatchNormalization(),
    GlobalMaxPool1D(),
    Dense(2048),
    LeakyReLU(0.8),
    Dropout(0.5),
    Dense(1024),
    LeakyReLU(0.8),
    Dropout(0.5),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(0.8),
    Dropout(0.5),
    BatchNormalization(),
    Dense(256),
    LeakyReLU(0.8),
    Dropout(0.5),
    BatchNormalization(),
    Dense(6, activation='softmax')
])

In [15]:
# init model checkpoint for saving the best model. Validation accuracy is monitored.
model_checkpoint_callback = ModelCheckpoint(
    filepath='final_model.h5',
    save_best_only=True,
    monitor='val_accuracy',
    verbose=1)

In [16]:
# compile model
model.compile(optimizer=Adamax(learning_rate=0.0001), loss="categorical_crossentropy", metrics='accuracy')

In [17]:
# fit data to model
model.fit(x=train_frames, y=train_class, batch_size=128, epochs=1000, validation_data=(val_frames, val_class), callbacks=model_checkpoint_callback)

Epoch 1/1000
3/3 [==============================] - ETA: 0s - loss: 2.4309 - accuracy: 0.1917
Epoch 1: val_accuracy improved from -inf to 0.17778, saving model to final_model.h5
3/3 [==============================] - 7s 334ms/step - loss: 2.4309 - accuracy: 0.1917 - val_loss: 1.8665 - val_accuracy: 0.1778
Epoch 2/1000
3/3 [==============================] - ETA: 0s - loss: 2.3298 - accuracy: 0.1917
Epoch 2: val_accuracy did not improve from 0.17778
3/3 [==============================] - 0s 58ms/step - loss: 2.3298 - accuracy: 0.1917 - val_loss: 1.8297 - val_accuracy: 0.1722
Epoch 3/1000
3/3 [==============================] - ETA: 0s - loss: 2.2871 - accuracy: 0.1944
Epoch 3: val_accuracy did not improve from 0.17778
3/3 [==============================] - 0s 49ms/step - loss: 2.2871 - accuracy: 0.1944 - val_loss: 1.8242 - val_accuracy: 0.1667
Epoch 4/1000
3/3 [==============================] - ETA: 0s - loss: 2.3780 - accuracy: 0.1722
Epoch 4: val_accuracy did not improve from 0.17778
3/

In [18]:
# load the best model
model = load_model('final_model.h5')

In [19]:
# evaluate model
model.evaluate(val_frames, val_class)

6/6 [==============================] - 0s 11ms/step - loss: 2.1330 - accuracy: 0.2167


[2.1330065727233887, 0.21666666865348816]

In [20]:
# change target data to integers
val_test = np.argmax(val_class, axis=1)

In [21]:
# find the highest probability and save as integer
val_pred = np.argmax(model.predict(val_frames), axis=1)

6/6 [==============================] - 0s 6ms/step


In [22]:
# check results for every class
print(classification_report(val_test, val_pred))

              precision    recall  f1-score   support

           0       0.25      0.07      0.11        30
           1       0.16      0.20      0.18        30
           2       0.24      0.37      0.29        30
           3       0.21      0.23      0.22        30
           4       0.23      0.23      0.23        30
           5       0.24      0.20      0.22        30

    accuracy                           0.22       180
   macro avg       0.22      0.22      0.21       180
weighted avg       0.22      0.22      0.21       180



In [23]:
# add validation data to train data
frames = np.append(train_frames, val_frames, axis=0)

In [24]:
# add validation targets to train targets
classes = np.append(train_class, val_class, axis=0)

In [25]:
# moke model checkpoint for model wihich will be additional trained. Loss is monitored.
model_checkpoint_callback = ModelCheckpoint(
    filepath='final_model_additional_train.h5',
    save_best_only=True,
    monitor='loss',
    verbose=1)

In [26]:
# train model again
model.fit(frames, classes, batch_size=128, epochs=1000, callbacks=model_checkpoint_callback)

Epoch 1/1000
5/5 [==============================] - ETA: 0s - loss: 2.0285 - accuracy: 0.2204
Epoch 1: loss improved from inf to 2.02848, saving model to final_model_additional_train.h5
5/5 [==============================] - 2s 108ms/step - loss: 2.0285 - accuracy: 0.2204
Epoch 2/1000
5/5 [==============================] - ETA: 0s - loss: 2.0023 - accuracy: 0.2278
Epoch 2: loss improved from 2.02848 to 2.00226, saving model to final_model_additional_train.h5
5/5 [==============================] - 0s 79ms/step - loss: 2.0023 - accuracy: 0.2278
Epoch 3/1000
3/5 [=================>............] - ETA: 0s - loss: 2.1040 - accuracy: 0.2500
Epoch 3: loss did not improve from 2.00226
5/5 [==============================] - 0s 27ms/step - loss: 2.1056 - accuracy: 0.2278
Epoch 4/1000
3/5 [=================>............] - ETA: 0s - loss: 1.9941 - accuracy: 0.2109
Epoch 4: loss did not improve from 2.00226
5/5 [==============================] - 0s 28ms/step - loss: 2.0360 - accuracy: 0.2074
Epoch